In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
#df_info = df_info[df_info.id_person < 50]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))

# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_time = int(np.max(df_info['len']))
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
np.unique(df_info.id_person.values)

array([ 17,  18,  19,  22,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  39,  40,  41,  42,  43,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107], dtype=int64)

In [3]:
df_info[df_info.id_person == 30]

,id,len,mean,std,id_person,id_location,id_direction,id_exp,id_paper,file,name,id_sign,median,max
5951,S030_1_1_1,14132.0,23.177852,6.752109,30,1,1,1,IMG_0708,IMG_0708_1.png,변준,30,22.044620,45.185267
5952,S030_1_1_1,14132.0,23.177852,6.752109,30,1,1,1,IMG_0708,IMG_0708_2.png,변준,30,22.044620,45.185267
5953,S030_1_1_1,14132.0,23.177852,6.752109,30,1,1,1,IMG_0708,IMG_0708_3.png,변준,30,22.044620,45.185267
5954,S030_1_1_1,14132.0,23.177852,6.752109,30,1,1,1,IMG_0708,IMG_0708_4.png,변준,30,22.044620,45.185267
5955,S030_1_1_10,14028.0,23.385705,6.708499,30,1,1,10,IMG_0708,IMG_0708_1.png,변준,30,22.637538,48.001735
5956,S030_1_1_10,14028.0,23.385705,6.708499,30,1,1,10,IMG_0708,IMG_0708_2.png,변준,30,22.637538,48.001735
5957,S030_1_1_10,14028.0,23.385705,6.708499,30,1,1,10,IMG_0708,IMG_0708_3.png,변준,30,22.637538,48.001735
5958,S030_1_1_10,14028.0,23.385705,6.708499,30,1,1,10,IMG_0708,IMG_0708_4.png,변준,30,22.637538,48.001735
5959,S030_1_1_11,13165.0,23.440472,6.725272,30,1,1,11,IMG_0708,IMG_0708_1.png,변준,30,22.718961,48.584627
5960,S030_1_1_11,13165.0,23.440472,6.725272,30,1,1,11,IMG_0708,IMG_0708_2.png,변준,30,22.718961,48.584627


In [50]:
file = 'S094_1_1_4'

In [51]:
with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
    data1 = pickle.load(f)

In [52]:
data1.shape

(14968, 30, 2, 3)

In [53]:
data_hc = np.load(path_csi_hc+'dataset_1_1.npy')

In [54]:
data1_hc = data_hc[100][4:]
data1_hc_lab = data_hc[100][:4]

In [55]:
data1_hc.shape

(90000,)

In [56]:
data1_hc_lab

array([94.,  1.,  1.,  4.])

In [57]:
data1_hc_resh = data1_hc.reshape([500,30,2,3])

In [58]:
np.mean(data1_hc_resh)

26.105434199702103

In [59]:
data1_abs = np.abs(data1)

In [63]:
target = data1_hc_resh[2].astype('int')
target

array([[[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 29, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 29]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 29, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28, 28, 28]],

       [[28, 28, 28],
        [28,

In [61]:
#data_array = data1_hc_resh
data_array = data1_abs
#for j in range(500):
for i in range(len(data_array)):
    if (data_array[i].astype('int') == target).all():
        print(i)

In [34]:
(data_array[0] == target).all()

True

In [67]:
data1_hc_resh

array([[[[28.7846115 , 28.69002752, 28.87211653],
         [28.81340733, 28.70089348, 28.77092839]],

        [[28.67869555, 28.81205149, 28.79566772],
         [28.82193441, 28.71761862, 28.7581616 ]],

        [[28.65659649, 28.77480931, 28.67036585],
         [28.61640281, 28.60771513, 28.64088687]],

        ...,

        [[29.82243495, 29.79589474, 29.77550827],
         [29.755063  , 30.05609409, 29.73961166]],

        [[29.86670208, 30.07628984, 30.15329593],
         [30.01147964, 29.90078735, 30.26993891]],

        [[30.27512069, 29.79146497, 29.80867734],
         [29.60531685, 29.55718391, 29.25730114]]],


       [[[29.07944206, 29.32684709, 29.29345984],
         [29.41130858, 29.62088575, 29.63642963]],

        [[29.81440535, 29.71104703, 29.62948181],
         [29.69878937, 29.43462943, 29.52901755]],

        [[29.33174821, 29.38104941, 29.22965312],
         [29.26245951, 29.29457683, 29.45897411]],

        ...,

        [[28.74779095, 28.81587999, 28.68512402],
  

In [68]:
data1_abs

array([[[[21.08701679, 27.13782479, 28.7804097 ],
         [29.54787033, 27.55614721, 26.11032696]],

        [[26.07220039, 24.10969481, 23.72147425],
         [22.57551616, 22.15295232, 21.24368326]],

        [[19.55309042, 19.57851157, 18.36374407],
         [18.46278925, 17.51344655, 17.31350394]],

        ...,

        [[13.91576796, 14.75990084, 15.11507707],
         [15.01603291, 15.20257463, 15.14794788]],

        [[15.81198625, 15.2461351 , 15.99961031],
         [16.06166463, 16.78843517, 15.81198625]],

        [[17.57958875, 17.31350394, 17.51344655],
         [16.16455911, 13.65113975, 11.53115849]]],


       [[[19.15613136, 24.80643626, 26.73300632],
         [25.70064715, 26.53350348, 25.30347582]],

        [[23.46491205, 22.16958305, 22.41538134],
         [20.77774173, 19.44006884, 17.92934179]],

        [[18.31417478, 17.89225934, 16.80103863],
         [15.7291337 , 15.02730579, 15.51656807]],

        ...,

        [[13.1662446 , 13.87844039, 14.23284059],
  